<a href="https://colab.research.google.com/github/ai4ai-lab/Reward-Shaping/blob/main/Lifegate_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import os
from google.colab import drive
import pickle
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from scipy.optimize import minimize
import random
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import torch

# deadend dependencies

In [2]:
!git clone https://github.com/microsoft/med-deadend.git


Cloning into 'med-deadend'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 130 (delta 52), reused 77 (delta 22), pack-reused 0
Receiving objects: 100% (130/130), 395.48 KiB | 10.69 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# shaping dependencies

In [3]:
!git clone https://github.com/ajagota7/Shaping.git

Cloning into 'Shaping'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 63 (delta 24), reused 55 (delta 18), pack-reused 0
Receiving objects: 100% (63/63), 11.54 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [4]:
# cd /content/

In [5]:
# %cd /content/Shaping

import zipfile

with zipfile.ZipFile('/content/Shaping/lifegate_1.zip', 'r') as zip_ref:
    # zip_ref.extractall('/content/med-deadend/lifegate/results/lifegate_1')
    zip_ref.extractall('/content/Shaping/')

In [6]:
%cd /content/med-deadend/lifegate


# results_dir = 'results/lifegate_1/'
results_dir = '/content/Shaping/'
# Load the Q tables from the primary learning agent, Q_D and Q_R value functions
with open(results_dir+'tabular_qnet.pkl', 'rb') as fq:
    ai = pickle.load(fq)

with open(results_dir+'tabular_qd.pkl', 'rb') as fd:
    ai_d = pickle.load(fd)

with open(results_dir+'tabular_qr.pkl', 'rb') as fr:
    ai_r = pickle.load(fr)

/content/med-deadend/lifegate


In [7]:
q_table = np.zeros((10, 10, 5))
q_d = np.zeros_like(q_table)
q_r = np.zeros_like(q_table)


for i in range(10):
    for j in range(10):
        for a in range(5):
            key = tuple([j, i, a])
            try:
                q_table[i,j,a] = ai.q[key]
                q_d[i,j,a] = ai_d.q[key]
                q_r[i,j,a] = ai_r.q[key]
            except:
                pass

In [8]:
import yaml
import random

In [9]:
from lifegate import LifeGate

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [10]:
params = yaml.safe_load(open('config.yaml', 'r'))

In [11]:
np.random.seed(seed=params['random_seed'])
random.seed(params['random_seed'])
random_state = np.random.RandomState(params['random_seed'])

In [12]:
env = LifeGate(max_steps=params['episode_max_len'], state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.4)

In [13]:
import Shaping

In [14]:
from Shaping import *

In [15]:
%cd /content/Shaping

from choose_actions import action_probs_top_n_epsilon
from shaping_features import *
from gen_policies import *
from IS import *
from subset_policies import *
from v_pi_e import *
from optimization import *
from neural_net import *
from prep_variance import *
from SCOPE_variance import SCOPE_variance

/content/Shaping


# Generating policies

In [16]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env, P_pi_b)

In [17]:
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env, P_pi_e)

# Prep

In [20]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [21]:
test1 = SCOPE_variance(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [22]:
samples_IS, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor = test1.prepare()

/content/Shaping/SCOPE_variance.py:89: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = self.dtype)


In [24]:
def clamp_large_terms(tensor):
  clamped_tensor = torch.clamp(tensor, min = -1e38, max = 1e38)
  return clamped_tensor

In [32]:
def clamp_large_terms(tensor):
  return tensor

In [25]:
states_output, states_first_output, states_last_output = test1.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
sums_states_weight_diff = test1.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
gamma_weights_states_last_sub_states_first = test1.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)
sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first = test1.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first)

In [39]:
# Print the min and max values of tensors
print("padded_weight_diff_tensors - Min:", padded_weight_diff_tensors.min().item(), " Max:", padded_weight_diff_tensors.max().item())
print("sums_states_weight_diff - Min:", sums_states_weight_diff.min().item(), " Max:", sums_states_weight_diff.max().item())
print("gamma_weights_last_tensor - Min:", gamma_weights_last_tensor.min().item(), " Max:", gamma_weights_last_tensor.max().item())
print("samples_IS - Min:", samples_IS.min().item(), " Max:", samples_IS.max().item())


padded_weight_diff_tensors - Min: -7.550129734505424e+42  Max: 6.991006254054227e+42
sums_states_weight_diff - Min: -3.3466686316947584e+17  Max: 5.410147133435166e+41
gamma_weights_last_tensor - Min: 0.0  Max: 9.232375374691587e+41
samples_IS - Min: -1.025819486076843e+42  Max: 1.0028234611748854e+17


In [62]:
def calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first):

  # states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)

  # Begin calcs without clamping

  # IS
  E_IS_sq = torch.mean(torch.mean(samples_IS, dim = 1)**2)
  E_IS_all_sq = torch.mean(torch.mean(samples_IS, dim = 1))**2

  # states_weight_diff
  E_s_wdiff_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim =1)**2)
  E_s_wdiff_all_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim = 1))**2

  # all terms
  SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first
  E_IS_SCOPE = torch.mean(torch.mean(samples_IS, dim =1) * torch.mean(SCOPE, dim =1))
  E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

  SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_IS_E_SCOPE

  IS_variance = E_IS_sq - E_IS_all_sq

  return IS_variance, SCOPE_variance


In [63]:
IS_variance, SCOPE_variance = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first)

In [64]:
SCOPE_variance

tensor(7.0500e+77, dtype=torch.float64, grad_fn=<SubBackward0>)

In [65]:
IS_variance

tensor(1.0786e+78, dtype=torch.float64)

# Optimizing

In [40]:
import torch.optim as optim

In [60]:
def train_var(model, num_epochs, learning_rate, samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0
        _, variance_loss = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first)

        print(f"Epoch {epoch+1}")
        print("Var loss: ", variance_loss)

        tot = variance_loss

        optimizer.zero_grad()
        tot.backward(retain_graph=True)
        optimizer.step()

        total_loss += tot.item()

        print(f"Total Loss: {total_loss}")
        print("-" * 40)

    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"Parameter name: {name}")
            print(f"Weights: {param.data}")

    return model


In [59]:
model2 = train_var(model, 10, 0.01, samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first)

Epoch 1
Var loss:  tensor(7.0500e+77, dtype=torch.float64, grad_fn=<SubBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

# Working on prep

In [22]:
# # Working on padding trajectories to allow for easier optimization

# def pad_trajectories(pi_b):
#     # Find the maximum length among all trajectories
#     max_length = max(len(traj) for traj in pi_b)

#     # Define the padding value
#     padding_value = np.array([np.array([0, 0]), 0, 0, np.array([0, 0]), 0, 0], dtype=object)

#     # Pad each trajectory to match the maximum length
#     padded_pi_b = [traj + [padding_value] * (max_length - len(traj)) for traj in pi_b]

#     return padded_pi_b

In [23]:
# pi_b_padded = pad_trajectories(pi_b)

In [24]:
def variance_terms_tens(eval_policy, behav_policy, behavior_policies):
  # Initialize lists to store axis data for each policy
  timesteps = []
  states = []
  state_first = []
  state_last = []
  actions = []
  rewards = []
  gamma_last = []
  weight_last = []
  weights = calculate_importance_weights(eval_policy, behav_policy, behavior_policies)
  psi = []

  for index, policy in enumerate(behavior_policies):
      policy_array = np.array(policy)
      timesteps.append(policy_array[:, 4].astype(int))
      # s.append(policy_array[:, 0])

      # last timestep for gamma
      gamma_last.append(len(policy))
      # last importance weight
      weight_last.append(weights[index][-1])


      states.append(policy_array[:, 0][1:])
      psi.append(policy_array[:,5][1:])
      state_first.append(policy_array[:,0][0])
      state_last.append(policy_array[:,0][-1])
      actions.append(policy_array[:, 1])
      rewards.append(policy_array[:, 2].astype(float))

  weights_difference = []
  for index, weight in enumerate(weights):
    # diff = np.array(w[index][:-1]) - np.array(w[index][1:])
    diff = np.array(weight[:-1]) - np.array(weight[1:])
    weights_difference.append(diff)

  return timesteps, states, state_first, state_last, actions, rewards, gamma_last, weight_last, weights, weights_difference

In [25]:
# t, s, s_f, s_l, a, r, g_l, w_l, w, w_diff = variance_terms_tens(P_pi_e, P_pi_b, pi_b)

In [26]:
timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = variance_terms_tens(P_pi_e, P_pi_b, pi_b)

In [27]:
import numpy as np

def padding_IS_terms(timesteps, actions, rewards, weights):
    # Find the maximum length among all lists
    max_length = max(len(traj) for traj in timesteps)

    # Define the padding values
    zero_padding = 0

    # Pad each list to match the maximum length
    padded_timesteps = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in timesteps]
    padded_rewards = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in rewards]
    padded_actions = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in actions]
    padded_weights = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights]

    return padded_timesteps, padded_rewards, padded_actions, padded_weights

In [28]:
padded_timesteps, padded_rewards, padded_actions, padded_weights = padding_IS_terms(timesteps, actions, rewards, weights)

In [29]:
def padding_states_weights_difference(states, weights_difference):
    # Find the maximum length of trajectories
    max_length = max(len(trajectory) for trajectory in states)

    zero_padding = 0

    # Pad each trajectory to make them all the same length
    padded_states = [
        [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
        for trajectory in states
    ]

    padded_weights_difference = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights_difference]

    return padded_states, padded_weights_difference

In [30]:
padded_states, padded_weights_difference = padding_states_weights_difference(states, weights_difference)

In [31]:
def tensorize_padded_terms(padded_states, padded_weights_difference):
  padded_state_tensors = torch.tensor(padded_states, dtype = torch.float64)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = torch.float64)
  padded_weight_diff_tensors = padded_weight_diff_tensors.unsqueeze(-1)

  return padded_state_tensors, padded_weight_diff_tensors


In [32]:
padded_state_tensors, padded_weight_diff_tensors = tensorize_padded_terms(padded_states, padded_weights_difference)

<ipython-input-31-3c22136e8020>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = torch.float64)


In [33]:
def tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last):
  states_first_tensor = torch.tensor(states_first, dtype = torch.float64)
  states_last_tensor = torch.tensor(states_last, dtype = torch.float64)
  gamma_last_tensor = torch.tensor(gamma_last, dtype = torch.float64)
  weights_last_tensor = torch.tensor(weights_last, dtype = torch.float64)

  return states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor

In [34]:
states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)

In [35]:
def calc_IS_terms(gamma, timesteps, rewards, weights):
  gtrw = np.power(gamma, timesteps)*rewards*weights

  IS_tensor = torch.sum(torch.tensor(gtrw, dtype = torch.float32), dim = 1, keepdim = True)

  return IS_tensor


In [36]:
IS_tensor = calc_IS_terms(0.9, padded_timesteps, padded_rewards, padded_weights)

In [37]:
torch.finfo(torch.float64).max

1.7976931348623157e+308

In [38]:
# samples_IS[0]

In [39]:
def calc_gamma_weight_last(gamma, gamma_last, weights_last):
  gamma_weight_last = np.power(gamma, gamma_last)*weights_last

  gamma_weight_last_tensor = torch.tensor(gamma_weight_last, dtype = torch.float64).unsqueeze(-1)

  return gamma_weight_last_tensor

In [40]:
gamma_weights_last_tensor = calc_gamma_weight_last(0.9, gamma_last, weights_last)

In [41]:
def bootstrap_IS_terms(IS_tensor, num_samples):
  seed = 42
  torch.manual_seed(seed)

  num_bootstraps = num_samples*len(IS_tensor)

  # Sample indices with replacement
  sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

  # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
  new_size = (num_samples, IS_tensor.shape[0])

  IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

  # sampled_tensor = IS_bootstraps.view(new_size)

  return IS_bootstraps

In [42]:
samples_IS = bootstrap_IS_terms(IS_tensor, 10000)

In [43]:
def clamp_large_terms(input_term):
  max_float64 = torch.finfo(torch.float64).max
  return torch.clamp(input_term, min=-max_float64, max=max_float64)


In [44]:
def clamp_large_terms(input_term):
    max_float64 = torch.tensor(torch.finfo(torch.float64).max, dtype=torch.float64)
    min_value = torch.tensor(-1e50, dtype=torch.float64)
    return torch.clamp(input_term, min=min_value, max=max_float64)

    # return torch.clamp(input_term, min=-max_float64, max=max_float64)


In [45]:
import torch.nn as nn
import torch.nn.functional as F

class CustomizableFeatureNet_d(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_prob=0.2, dtype=torch.float32):
        super(CustomizableFeatureNet_d, self).__init__()
        self.hidden_layers = nn.ModuleList()

        # Create the hidden layers based on the provided sizes
        for in_dim, out_dim in zip([input_dim] + hidden_dims, hidden_dims):
            self.hidden_layers.append(nn.Linear(in_dim, out_dim).to(dtype))

        self.output_layer = nn.Linear(hidden_dims[-1], output_dim).to(dtype)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [46]:
model = CustomizableFeatureNet_d(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [47]:
# model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [48]:
def pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor):
  # Get model outputs for states
  states_output = model(padded_state_tensors)
  states_first_output = model(states_first_tensor)
  states_last_output = model(states_last_tensor)
  return states_output, states_first_output, states_last_output

In [49]:
states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)

In [50]:
def states_weight_diff_sums(states_output, padded_weight_diff_tensors):
  states_weight_diff = states_output * padded_weight_diff_tensors
  sums_states_weight_diff = torch.sum(states_weight_diff, dim =1)

  return sums_states_weight_diff

In [51]:
sums_states_weight_diff = states_weight_diff_sums(states_output, padded_weight_diff_tensors)

In [52]:
def last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output):
  gamma_weights_states_last_sub_states_first = gamma_weights_last_tensor*states_last_output -  states_first_output

  return gamma_weights_states_last_sub_states_first

In [53]:
gamma_weights_states_last_sub_states_first = last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)

In [54]:
# gamma_weights_states_last_sub_states_first.shape

In [55]:
def bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, num_samples):

  seed = 42
  torch.manual_seed(seed)

  num_bootstraps = num_samples*sums_states_weight_diff.shape[0]

  # Sample indices with replacement
  sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps,), dtype=torch.long)

  # sizes
  # size_states_weights_diff = (num_samples, states_output.shape[0], states_output.shape[1])
  reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

  # Resize samples to shape num_samples x num_trajectories x length_padded_trajectories
  # samples_states_output = states_output[sampled_indices].view(size_states_weights_diff)
  # samples_weight_diff = padded_weight_diff_tensors[sampled_indices].view(size_states_weights_diff)

  # Resize samples to shape num_samples x num_trajectories
  sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
  # samples_states_first_output = sums_states_weight_diff[sampled_indices].view(reshaped_size)
  # samples_states_last_output = states_last_output[sampled_indices].view(reshaped_size)
  samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)


  return sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first


In [56]:
sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first = bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, 10000)

In [57]:
# def gtrw_plus_states_weight_diff_min_last_firsts_terms(gtrw_tensor, sums_states_weight_diff, gamma_weights_states_last_sub_states_first):
#   sum_IS_and_shaping = gtrw_tensor + sums_states_weight_diff + gamma_weights_states_last_sub_states_first

#   return sum_IS_and_shaping

In [58]:
# def bootstrap_IS_shaping_terms(gtrw_tensor, sums_states_weight_diff, gamma_weights_states_last_sub_states_first):
#   seed = 42
#   torch.manual_seed(seed)

#   num_bootstraps = num_samples*sums_states_weight_diff.shape[0]

#   # Sample indices with replacement
#   sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps,), dtype=torch.long)



In [59]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim = 1))))

In [60]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim =1)**2)))

In [61]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim =1))**2) - torch.mean(clamp_large_terms(torch.mean(samples_IS, dim = 1))))

In [62]:
# SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first

In [63]:
# torch.mean(SCOPE,dim =1).shape

In [64]:
torch.mean(clamp_large_terms(samples_IS), dim = 1)

tensor([      -inf, 2.0056e+14,       -inf,  ...,       -inf,       -inf,
        1.0028e+14])

In [65]:
def calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first):


  # states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)



  # Begin calcs without clamping

  # IS
  E_IS_sq = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1)**2)
  E_IS_all_sq = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1))**2

  # states_weight_diff
  E_s_wdiff_sq = torch.mean(torch.mean(clamp_large_terms(sample_sums_states_weight_diff), dim =1)**2)
  E_s_wdiff_all_sq = torch.mean(torch.mean(clamp_large_terms(sample_sums_states_weight_diff), dim = 1))**2

  # all terms
  SCOPE = clamp_large_terms(sample_sums_states_weight_diff)+clamp_large_terms(samples_gamma_weight_states_last_sub_states_first)
  E_IS_SCOPE = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim =1) * torch.mean(SCOPE, dim =1))
  E_IS_E_SCOPE = torch.mean(torch.mean(clamp_large_terms(samples_IS),dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

  SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_IS_E_SCOPE

  IS_variance = E_IS_sq - E_IS_all_sq

  return IS_variance, SCOPE_variance


In [66]:
IS_variance, SCOPE_variance = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first)

In [67]:
torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1)**2)

tensor(inf)

In [68]:
SCOPE_variance

tensor(nan, dtype=torch.float64, grad_fn=<SubBackward0>)

In [69]:
IS_variance

tensor(nan)

In [70]:
import torch

def clamp_large_terms(input_term):
    max_float64 = torch.tensor(torch.finfo(torch.float64).max, dtype=torch.float64)
    # min_value = torch.tensor(-1e38, dtype=torch.float64)
    min_value = torch.tensor(-9e-38, dtype=torch.float64)

    # min_value = torch.tensor(-1.7e+308, dtype = torch.float64)
    # min_value = torch.tensor(torch.finfo(torch.float64).min, dtype=torch.float64)


    # Using torch.where to explicitly set values outside the desired range
    clamped_result = torch.where(input_term < min_value, min_value, input_term)
    clamped_result = torch.where(clamped_result > max_float64, max_float64, clamped_result)

    return clamped_result
clamp_large_terms(samples_IS)[0].min()

tensor(-9.0000e-38)

In [71]:
samples_IS[0]

tensor([ 0.0000e+00, -9.5567e-34, -1.1020e-12, -5.7703e-20,  0.0000e+00,
        -6.6609e-06,  0.0000e+00,  0.0000e+00,  0.0000e+00, -7.4562e-17,
        -1.3518e-33, -6.3068e-29, -1.1566e-08,  0.0000e+00, -3.2522e-24,
         0.0000e+00,  0.0000e+00, -4.0924e-02,  0.0000e+00,  0.0000e+00,
        -6.0833e-21, -1.1020e-12,  0.0000e+00, -3.5032e-43, -3.4013e-37,
        -7.9832e-23,  0.0000e+00, -2.2156e-12,  0.0000e+00, -7.3671e-27,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00, -3.0953e-12,  0.0000e+00, -7.1745e-11,  0.0000e+00,
        -1.2184e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -4.8370e-19,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -1.7865e-25, -3.0953e-12, -3.1972e-14,
         0.0000e+00, -3.9939e-21,  0.0000e+00, -4.2383e-17,  0.0000e+00,
         0.0000e+00, -9.5567e-34,  0.0000e+00,  0.0000e+00, -5.4262e-07,
        -3.7487e-37, -2.6165e-41,  0.0000e+00, -1.0

In [72]:
clamp_large_terms(samples_IS)[0].min()

tensor(-9.0000e-38)

In [73]:
def clamp_large_terms(tensor):
    max_float64 = torch.finfo(torch.float64).max
    min_float64 = torch.finfo(torch.float64).min

    tensor[tensor == float('inf')] = 1e38  # Choose a large finite value
    tensor[tensor == float('-inf')] = -1e38  # Choose a small finite value

    return tensor

# Example usage:
# samples_IS_clamped = clamp_tensor(samples_IS)

In [74]:
# samples_IS_clamped[0]

NameError: name 'samples_IS_clamped' is not defined

In [75]:
torch.tensor(torch.finfo(torch.float64).min, dtype=torch.float64)

tensor(-1.7977e+308, dtype=torch.float64)

In [76]:
model.eval()

CustomizableFeatureNet_d(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=32, bias=True)
  )
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [82]:
output = model(state_tensors)

RuntimeError: mat1 and mat2 must have the same dtype, but got Float and Double

In [ ]:
output.shape

In [ ]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
scope_set, phi_set = subset_policies(pi_b, 0.3)

In [ ]:
IS, state_tensors, w_diff, f, st_og, psi_res, sample_last_tensors = variance_terms_tens(P_pi_e, P_pi_b, phi_set)

In [ ]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
model2 = train_mse_var(model, 40, 0.001, 1, 1, IS, state_tensors, w_diff, f, sample_last_tensors, phi_set, st_og, psi_res)

# Test class

In [ ]:
class SCOPE_variance(object):
    def __init__(self, model, gamma, num_bootstraps, pi_b, P_pi_b, P_pi_e):
        self.model = model
        self.gamma = gamma
        self.num_bootstraps = num_bootstraps
        self.pi_b = pi_b
        self.P_pi_b = P_pi_b
        self.P_pi_e = P_pi_e




    def prep_policies(self):
        # Initialize lists to store axis data for each policy
        timesteps = []
        states = []
        state_first = []
        state_last = []
        actions = []
        rewards = []
        gamma_last = []
        weight_last = []
        weights = calculate_importance_weights(self.P_pi_e, self.P_pi_b, self.pi_b)
        psi = []

        for index, policy in enumerate(pi_b):
            policy_array = np.array(policy)
            timesteps.append(policy_array[:, 4].astype(int))
            # s.append(policy_array[:, 0])

            # last timestep for gamma
            gamma_last.append(len(policy))
            # last importance weight
            weight_last.append(weights[index][-1])


            states.append(policy_array[:, 0][1:])
            psi.append(policy_array[:,5][1:])
            state_first.append(policy_array[:,0][0])
            state_last.append(policy_array[:,0][-1])
            actions.append(policy_array[:, 1])
            rewards.append(policy_array[:, 2].astype(float))

        weights_difference = []
        for index, weight in enumerate(weights):
            # diff = np.array(w[index][:-1]) - np.array(w[index][1:])
            diff = np.array(weight[:-1]) - np.array(weight[1:])
            weights_difference.append(diff)

        return timesteps, states, state_first, state_last, actions, rewards, gamma_last, weight_last, weights, weights_difference

    def padding_IS_terms(self,timesteps, actions, rewards, weights):
        # Find the maximum length among all lists
        max_length = max(len(traj) for traj in timesteps)

        # Define the padding values
        zero_padding = 0

        # Pad each list to match the maximum length
        padded_timesteps = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in timesteps]
        padded_rewards = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in rewards]
        padded_actions = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in actions]
        padded_weights = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights]

        return padded_timesteps, padded_rewards, padded_actions, padded_weights

    def padding_states_weights_difference(self, states, weights_difference):
        # Find the maximum length of trajectories
        max_length = max(len(trajectory) for trajectory in states)

        zero_padding = 0

        # Pad each trajectory to make them all the same length
        padded_states = [
            [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
            for trajectory in states
        ]

        padded_weights_difference = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights_difference]

        return padded_states, padded_weights_difference

    def tensorize_padded_terms(self, padded_states, padded_weights_difference):
        padded_state_tensors = torch.tensor(padded_states, dtype = torch.float32)
        padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = torch.float32)
        padded_weight_diff_tensors = padded_weight_diff_tensors.unsqueeze(-1)

        return padded_state_tensors, padded_weight_diff_tensors

    def tensorize_last_and_first_terms(self, states_first, states_last, gamma_last, weights_last):
        states_first_tensor = torch.tensor(states_first, dtype = torch.float32)
        states_last_tensor = torch.tensor(states_last, dtype = torch.float32)
        gamma_last_tensor = torch.tensor(gamma_last, dtype = torch.float32)
        weights_last_tensor = torch.tensor(weights_last, dtype = torch.float32)

        return states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor

    def calc_IS_terms(self, gamma, timesteps, rewards, weights):
        gtrw = np.power(gamma, timesteps)*rewards*weights

        IS_tensor = torch.sum(torch.tensor(gtrw, dtype = torch.float32), dim = 1, keepdim = True)

        return IS_tensor

    def calc_gamma_weight_last(self, gamma, gamma_last, weights_last):
        gamma_weight_last = np.power(gamma, gamma_last)*weights_last

        gamma_weight_last_tensor = torch.tensor(gamma_weight_last, dtype = torch.float32).unsqueeze(-1)

        return gamma_weight_last_tensor
    def bootstrap_IS_terms(self, IS_tensor, num_samples):
        seed = 42
        torch.manual_seed(seed)

        num_bootstraps = num_samples*len(IS_tensor)

        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

        # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
        new_size = (num_samples, IS_tensor.shape[0])

        IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

        # sampled_tensor = IS_bootstraps.view(new_size)

        return IS_bootstraps

    def states_weight_diff_sums(self, states_output, padded_weight_diff_tensors):
        states_weight_diff = states_output * padded_weight_diff_tensors
        sums_states_weight_diff = torch.sum(states_weight_diff, dim =1)

        return sums_states_weight_diff

    def bootstrap_shaping_terms(self, sums_states_weight_diff, gamma_weights_states_last_sub_states_first, num_samples):

        seed = 42
        torch.manual_seed(seed)

        num_bootstraps = num_samples*sums_states_weight_diff.shape[0]

        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps,), dtype=torch.long)

        reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

        # Resize samples to shape num_samples x num_trajectories
        sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
        samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)


        return sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first

    def pass_states(self,model, padded_state_tensors, states_first_tensor, states_last_tensor):
        # Get model outputs for states
        states_output = model(padded_state_tensors)
        states_first_output = model(states_first_tensor)
        states_last_output = model(states_last_tensor)
        return states_output, states_first_output, states_last_output


    def prepare(self):
        timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = self.prep_policies()
        padded_timesteps, padded_rewards, padded_actions, padded_weights = self.padding_IS_terms(timesteps, actions, rewards, weights)
        padded_states, padded_weights_difference = self.padding_states_weights_difference(states, weights_difference)
        padded_state_tensors, padded_weight_diff_tensors = self.tensorize_padded_terms(padded_states, padded_weights_difference)
        states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = self.tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)
        IS_tensor = self.calc_IS_terms(self.gamma, padded_timesteps, padded_rewards, padded_weights)
        gamma_weights_last_tensor = self.calc_gamma_weight_last(self.gamma, gamma_last, weights_last)
        samples_IS = self.bootstrap_IS_terms(IS_tensor, self.num_bootstraps)

        return samples_IS, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor


In [78]:
test1 = SCOPE_variance(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e)

In [ ]:
s,p_s, p_w, g_w_l, s_f, s_l = test1.prepare()